In [1]:
import pandas as pd 
import numpy as np 

from sklearn.model_selection import train_test_split

from tqdm import tqdm

from sklearn.metrics import accuracy_score, f1_score

In [2]:
train = pd.read_csv('../data/processed/train.csv')
test = pd.read_csv('../data/processed/test.csv')
print('Train: ',train.shape)
print('Test: ',test.shape)

Train:  (67447, 43)
Test:  (18816, 42)


train_ngram_features = pd.read_csv('../data/features/ngram/5_ngram_train.csv')
test_ngram_features = pd.read_csv('../data/features/ngram/5_ngram_test.csv')

print('Train: ',train_ngram_features.shape)
print('Test: ',test_ngram_features.shape)

In [3]:
FOLD = 10
df = pd.DataFrame()
for f_idx in tqdm(range(1,FOLD+1)):
    path = '../data/features/blast/'+str(FOLD)+'/dev_result_'+str(f_idx)+'.csv'
    new_df = pd.read_csv(path)
    df = pd.concat([df, new_df], ignore_index=True,axis=0)
del new_df
print(df.shape)

100%|██████████| 10/10 [06:08<00:00, 36.85s/it]
(66739, 14455)


In [4]:
df.sequence_id.duplicated().any()

False

In [5]:
%%time
#df = df.groupby('sequence_id', as_index=False).mean()
#print(df.shape)

Wall time: 998 µs


In [6]:
#00Q4V31Thits,00Q4V31Tidentity,00Q4V31Talignment length,00Q4V31Tmismatches,00Q4V31Tgap opens,00Q4V31Tq. start,00Q4V31Tq. end,00Q4V31Ts. start,00Q4V31Ts. end,00Q4V31Tevalue,00Q4V31Tbit score,
print(df.shape)
columnVals = df.columns.map(lambda x: ('hits' in x) | ('identity' in x) | ('alignment length' in x) | ('mismatches' in x) | ('gap opens' in x) | ('bit score in x' in x) | ('sequence_id' in x))
df = df.loc[:,columnVals]
df.shape

(66739, 14455)


(66739, 6571)

In [7]:
%%time
blast = pd.merge(train,df,how='left',on='sequence_id')
blast.shape

Wall time: 6.9 s


(67447, 6613)

In [8]:
blast.fillna(0,inplace=True)
#blast.dropna(inplace=True)

In [9]:
def top10_accuracy_scorer(estimator, X, y):

    #probas = estimator.predict_proba(X)
    
    top10_idx = np.argpartition(probas, -10, axis=1)[:, -10:]
    
    top10_preds = estimator.classes_[top10_idx]

    mask = top10_preds == np.reshape(np.array(y.values.ravel()),(y.shape[0],1))
    
    top_10_accuracy = mask.any(axis=1).mean()
 
    return top_10_accuracy

In [10]:
# Rename our feature array
#train.drop(['sequence','sequence_id'],inplace=True,axis=1)
blast.drop(['sequence_id','sequence'],inplace=True,axis=1)

In [11]:
from sklearn.preprocessing import OneHotEncoder
y = blast['target']
y2 = blast['target'].values.reshape(-1, 1)
target_encoder = OneHotEncoder()
target_encoder.fit(y2)

X = blast.drop('target',inplace=False,axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=420,stratify=y)

In [22]:
import torch

In [ ]:
X_train = torch.tensor(X_train.values.astype(np.float32)) 
y_train = torch.tensor(target_encoder.transform(y_train.values.reshape(-1, 1)).astype(np.float32)) 
train_dataset = torch.utils.data.TensorDataset(X_train, y_train) 
train_dataloader = torch.utils.data.DataLoader(dataset = train_dataset, 
    batch_size = 16, 
    sampler = torch.utils.data.RandomSampler(train_dataset)
)

In [ ]:
X_test = torch.tensor(X_test.values.astype(np.float32)) 
y_test = torch.tensor(target_encoder.transform(y_test.values.reshape(-1, 1)).astype(np.float32)) 
test_dataset = torch.utils.data.TensorDataset(X_test, y_test) 
test_dataloader = torch.utils.data.DataLoader(dataset = test_dataset, 
    batch_size = 32, 
    sampler = torch.utils.data.SequentialSampler(test_dataset)
)

In [ ]:
print(X_train.shape)
print(y_train.shape)

In [ ]:
class Net(torch.nn.Model):

    def __init__(self):
        super(self).__init__()

    
    def forward(self,inputs):


        

In [ ]:
epochs = 50

device = torch.device('cuda')
model = Net().to(device)

learning_rate = 2e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

criterion = torch.nn.CrossEntropyLoss()

In [16]:
for epoch_i in range(epochs):
    print('EPOCH: ',epoch_i)
    ground_truth = []
    preds = []
    total_loss=0
    print('Training...')
    for step, batch in tqdm(enumerate(train_dataloader)):

        inputs = batch[0].to(device)
        b_target = batch[1]

        b_logits = model(inputs)

        preds.extend(b_logits)
        ground_truth.extend(b_target)

        loss=criterion(b_target,b_logits)
        total_loss+=loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


    acc = accuracy_score(ground_truth,preds)
    f1 = f1_score(ground_truth,preds,average='macro')
    top = top10_accuracy_scorer(model, X_d, y_d)

    print('Loss: ',total_loss/len(train_dataloader))
    print('ACC: ',acc)
    print('F1: ', f1)
    print('TOP-10: ',top)

0%|          | 0/5 [00:00<?, ?it/s]
--------FOLD  1
 20%|██        | 1/5 [07:04<28:18, 424.59s/it]ACC:  0.47234988880652334
F1:  0.17043789934048822
TOP-10:  0.8400296515937732

--------FOLD  2
 20%|██        | 1/5 [07:49<31:16, 469.19s/it]


KeyboardInterrupt: 